In [2]:
!pip install -q torch_geometric
!pip install -q class_resolver
!pip3 install pymatting

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 31.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.7/54.7 kB 3.3 MB/s eta 0:00:00


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as nnFn
import torch.optim as optim
import numpy as np
import random
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from torch_geometric.data import Data
from torch_geometric.nn import ARMAConv
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    f1_score, roc_auc_score, log_loss
)
from torch.utils.data import TensorDataset, DataLoader, Subset
from torch_geometric.data import Data
from torch_geometric.nn import ARMAConv
from torch.utils.data import TensorDataset, DataLoader, Subset
from torchvision import models

In [5]:
import torch
print("CUDA available:", torch.cuda.is_available())
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("GPU Name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU")

CUDA available: True
GPU Name: Tesla T4


In [6]:
data = np.load('/content/drive/MyDrive/TejaswiAbburi_va797/Dataset/Medmnist_data/breastmnist_224.npz', allow_pickle=True)
all_images = np.concatenate([data['train_images'], data['val_images'], data['test_images']], axis=0)
all_labels = np.concatenate([data['train_labels'], data['val_labels'], data['test_labels']], axis=0).squeeze()

# Convert to 3-channel RGB
images = all_images.astype(np.float32) / 255.0
images = np.repeat(images[:, None, :, :], 3, axis=1)  # (N, 3, 224, 224)
X_torch = torch.tensor(images)
y_torch = torch.tensor(all_labels).long()
print(f"Raw images: {X_torch.shape}, Labels: {y_torch.shape}")

Raw images: torch.Size([780, 3, 224, 224]), Labels: torch.Size([780])


In [7]:
class0_idx = [i for i in range(len(y_torch)) if y_torch[i] == 0]
class1_idx = [i for i in range(len(y_torch)) if y_torch[i] == 1]

random.seed(42)
sampled_class0 = random.sample(class0_idx, min(1000, len(class0_idx)))
sampled_class1 = random.sample(class1_idx, min(1000, len(class1_idx)))

selected_indices = sampled_class0 + sampled_class1
random.shuffle(selected_indices)

subset_dataset = Subset(TensorDataset(X_torch, y_torch), selected_indices)
subset_loader = DataLoader(subset_dataset, batch_size=64, shuffle=False)

In [8]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
vit = torch.hub.load('facebookresearch/dino:main', 'dino_vitb16')
vit.eval().to(device)

vit_feats, y_list = [], []

with torch.no_grad():
    for imgs, lbls in subset_loader:
        imgs = imgs.to(device)
        feats = vit(imgs)
        vit_feats.append(feats.cpu())
        y_list.extend(lbls.cpu().tolist())

F = torch.cat(vit_feats, dim=0).numpy().astype(np.float32)
y = np.array(y_list).astype(np.int64)

num_nodes, num_feats = F.shape
print(f"Extracted ViT-DINO Features: {F.shape}, Labels: {y.shape}")

Downloading: "https://github.com/facebookresearch/dino/zipball/main" to /root/.cache/torch/hub/main.zip
Downloading: "https://dl.fbaipublicfiles.com/dino/dino_vitbase16_pretrain/dino_vitbase16_pretrain.pth" to /root/.cache/torch/hub/checkpoints/dino_vitbase16_pretrain.pth


100%|██████████| 327M/327M [00:01<00:00, 296MB/s]


Extracted ViT-DINO Features: (780, 768), Labels: (780,)


In [6]:
# resnet = models.resnet18(pretrained=True)
# resnet.fc = nn.Identity()
# resnet = resnet.to(device)
# resnet.eval()

# resnet_feats = []
# y_list = []
# with torch.no_grad():
#     for imgs, labels in subset_loader:
#         imgs = imgs.to(device)
#         feats = resnet(imgs)
#         resnet_feats.append(feats.cpu())
#         y_list.extend(labels.cpu().tolist())

# features = torch.cat(resnet_feats, dim=0).numpy().astype(np.float32)  # shape (N, feat_dim)
# y_labels = np.array(y_list).astype(np.float32)
# print("Feature shape:", features.shape, "Label shape:", y_labels.shape)

In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as nnFn
from torch_geometric.nn import ARMAConv

class ARMA_SemiSupervised(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, device, num_stacks=1, num_layers=1):
        super(ARMA_SemiSupervised, self).__init__()
        self.device = device

        self.conv1 = ARMAConv(input_dim, hidden_dim, num_stacks=num_stacks, num_layers=num_layers,
                              shared_weights=True, dropout=0.3)
        self.conv2 = ARMAConv(hidden_dim, hidden_dim, num_stacks=num_stacks, num_layers=num_layers,
                              shared_weights=True, dropout=0.3)
        self.conv3 = ARMAConv(hidden_dim, hidden_dim, num_stacks=num_stacks, num_layers=num_layers,
                              shared_weights=True, dropout=0.3)

        self.dropout = nn.Dropout(0.25)
        self.fc = nn.Linear(hidden_dim, output_dim)

        activations = {
            "SELU": nnFn.selu,
            "SiLU": nnFn.silu,
            "GELU": nnFn.gelu,
            "RELU": nnFn.relu,
            "ELU": nnFn.elu,
            "PReLU": nnFn.prelu,
            "LeakyReLU": nnFn.leaky_relu
        }
        self.act = activations.get("RELU", nnFn.relu)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        # --- Layer 1 ---
        x = self.conv1(x, edge_index)
        x = self.act(x)

        x = self.dropout(x)

        # --- Layer 2 ---
        x = self.conv2(x, edge_index)
        x = self.act(x)
        x = self.dropout(x)

        # --- Layer 3 ---
        x = self.conv3(x, edge_index)
        x = self.act(x)
        x = self.dropout(x)

        logits = self.fc(x)
        return logits

    def modularity_loss(self, A, logits):
        S = nnFn.softmax(logits, dim=1)
        d = torch.sum(A, dim=1)
        m = torch.sum(A)
        B = A - torch.outer(d, d) / (2 * m)

        modularity_term = (-1 / (2 * m)) * torch.trace(S.T @ B @ S)

        I_S = torch.eye(S.shape[1], device=self.device)
        k = torch.norm(I_S)
        n = S.shape[0]
        collapse_reg = (torch.sqrt(k) / n) * torch.norm(torch.sum(S, dim=0), p='fro') - 1
        entropy_reg = -torch.mean(torch.sum(S * torch.log(S + 1e-9), dim=1))

        return modularity_term + 0.1 * collapse_reg + 0.01 * entropy_reg


In [8]:
# import torch
# import torch.nn as nn
# import torch.nn.functional as nnFn
# from torch_geometric.nn import GCNConv

# class GCN_SemiSupervised(nn.Module):
#     def __init__(self, input_dim, hidden_dim, output_dim, device):
#         super(GCN_SemiSupervised, self).__init__()
#         self.device = device

#         # 3-layer GCN
#         self.conv1 = GCNConv(input_dim, hidden_dim)
#         self.conv2 = GCNConv(hidden_dim, hidden_dim)
#         self.conv3 = GCNConv(hidden_dim, hidden_dim)

#         self.dropout = nn.Dropout(0.25)
#         self.fc = nn.Linear(hidden_dim, output_dim)

#         # Activation function
#         activations = {
#             "SELU": nnFn.selu,
#             "SiLU": nnFn.silu,
#             "GELU": nnFn.gelu,
#             "RELU": nnFn.relu,
#             "ELU": nnFn.elu,
#             "PReLU": nnFn.prelu,
#             "LeakyReLU": nnFn.leaky_relu
#         }
#         self.act = activations.get("RELU", nnFn.relu)

#     def forward(self, data):
#         x, edge_index = data.x, data.edge_index

#         # --- Layer 1 ---
#         x = self.conv1(x, edge_index)
#         x = self.act(x)
#         x = self.dropout(x)

#         # --- Layer 2 ---
#         x = self.conv2(x, edge_index)
#         x = self.act(x)
#         x = self.dropout(x)

#         # --- Layer 3 ---
#         x = self.conv3(x, edge_index)
#         x = self.act(x)
#         x = self.dropout(x)

#         logits = self.fc(x)
#         return logits

#     def modularity_loss(self, A, logits):
#         S = nnFn.softmax(logits, dim=1)
#         d = torch.sum(A, dim=1)
#         m = torch.sum(A)
#         B = A - torch.outer(d, d) / (2 * m)

#         modularity_term = (-1 / (2 * m)) * torch.trace(S.T @ B @ S)

#         I_S = torch.eye(S.shape[1], device=self.device)
#         k = torch.norm(I_S)
#         n = S.shape[0]
#         collapse_reg = (torch.sqrt(k) / n) * torch.norm(torch.sum(S, dim=0), p='fro') - 1
#         entropy_reg = -torch.mean(torch.sum(S * torch.log(S + 1e-9), dim=1))

#         return modularity_term + 0.1 * collapse_reg + 0.01 * entropy_reg


In [10]:
def create_adj(F, alpha=1):
    F_norm = F / np.linalg.norm(F, axis=1, keepdims=True)
    W = np.dot(F_norm, F_norm.T)
    W = np.where(W >= alpha, 1, 0).astype(np.float32)
    W = W / W.max()
    return W

def load_data(adj, node_feats):
    node_feats = torch.from_numpy(node_feats).float()
    edge_index = torch.from_numpy(np.array(np.nonzero((adj > 0))))
    return Data(x=node_feats, edge_index=edge_index)

In [10]:
# device = 'cuda' if torch.cuda.is_available() else 'cpu'
# alpha = 0.78
# feats_dim = num_feats
# hidden_dim = 256
# num_classes = 2
# num_epochs = 2000
# lr = 0.0001
# weight_decay = 1e-4
# batch_print_freq = 100
# lambda_mod = 0.5 #0.001  # weight for modularity loss
# # lambda_sup = 5

In [11]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
alpha = 0.73
feats_dim = num_feats
hidden_dim = 256
num_classes = 2
num_epochs = 2000
lr = 0.0001
weight_decay = 1e-4
batch_print_freq = 100
lambda_mod =  0.001  #0.2 #0.001  # weight for modularity loss
# lambda_sup = 1.0

In [12]:
W = create_adj(F, alpha)
data = load_data(W, F).to(device)
print(data)

Data(x=[780, 768], edge_index=[2, 266936])


In [13]:
sss = StratifiedShuffleSplit(n_splits=20, test_size=0.9, random_state=42)

accuracies, precisions, recalls, f1_scores, aucs, ce_losses = [], [], [], [], [], []

A_tensor = torch.from_numpy(W).float().to(device)

for fold, (train_idx, test_idx) in enumerate(sss.split(F, y), start=1):
    print(f"\n=== Fold {fold} ===")

    train_idx_t = torch.from_numpy(train_idx).long().to(device)
    y_train_tensor = torch.from_numpy(y[train_idx]).long().to(device)

    model = ARMA_SemiSupervised(feats_dim, hidden_dim, num_classes, device).to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    ce_loss = nn.CrossEntropyLoss()

    # Training
    for epoch in range(1, num_epochs + 1):
        model.train()
        optimizer.zero_grad()

        logits = model(data)
        loss_sup = ce_loss(logits[train_idx_t], y_train_tensor)
        loss_unsup = model.modularity_loss(A_tensor, logits)
        total_loss = loss_sup + lambda_mod * loss_unsup

        total_loss.backward()
        optimizer.step()

        if epoch % batch_print_freq == 0 or epoch == 1:
            model.eval()
            with torch.no_grad():
                preds_train = logits[train_idx_t].argmax(dim=1)
                acc_train = accuracy_score(y_train_tensor.cpu(), preds_train.cpu())
            print(f"Fold {fold} Epoch {epoch}: "
                  f"TotalLoss={total_loss.item():.6f} | Sup={loss_sup.item():.6f} | "
                  f"Unsup={loss_unsup.item():.6f} | TrainAcc={acc_train:.4f}")

    # Evaluation
    model.eval()
    with torch.no_grad():
        out = model(data)
        preds = out.argmax(dim=1).cpu().numpy()
        probs = torch.softmax(out, dim=1)[:, 1].cpu().numpy()  # Probability for class 1

    y_test = y[test_idx]
    y_pred_test = preds[test_idx]
    y_prob_test = probs[test_idx]

    acc = accuracy_score(y_test, y_pred_test)
    prec = precision_score(y_test, y_pred_test, zero_division=0)
    rec = recall_score(y_test, y_pred_test, zero_division=0)
    f1 = f1_score(y_test, y_pred_test, zero_division=0)
    auc = roc_auc_score(y_test, y_prob_test)
    ce = log_loss(y_test, y_prob_test)

    accuracies.append(acc)
    precisions.append(prec)
    recalls.append(rec)
    f1_scores.append(f1)
    aucs.append(auc)
    ce_losses.append(ce)

    print(f"Fold {fold} → "
          f"Acc={acc:.4f} | Prec={prec:.4f} | Rec={rec:.4f} | "
          f"F1={f1:.4f} | AUC={auc:.4f} | CE Loss={ce:.4f}")


# Final summary
print("\n=== Average Results Across 20 Folds ===")
print(f"Accuracy:  {np.mean(accuracies):.4f} ± {np.std(accuracies):.4f}")
print(f"Precision: {np.mean(precisions):.4f} ± {np.std(precisions):.4f}")
print(f"Recall:    {np.mean(recalls):.4f} ± {np.std(recalls):.4f}")
print(f"F1-score:  {np.mean(f1_scores):.4f} ± {np.std(f1_scores):.4f}")
print(f"AUC:       {np.mean(aucs):.4f} ± {np.std(aucs):.4f}")
print(f"CE Loss:   {np.mean(ce_losses):.4f} ± {np.std(ce_losses):.4f}")


=== Fold 1 ===
Fold 1 Epoch 1: TotalLoss=1.012460 | Sup=1.012601 | Unsup=-0.140813 | TrainAcc=0.5641
Fold 1 Epoch 100: TotalLoss=0.363546 | Sup=0.363716 | Unsup=-0.170727 | TrainAcc=0.8718
Fold 1 Epoch 200: TotalLoss=0.201759 | Sup=0.201946 | Unsup=-0.186731 | TrainAcc=0.9231
Fold 1 Epoch 300: TotalLoss=0.136428 | Sup=0.136606 | Unsup=-0.178382 | TrainAcc=0.9872
Fold 1 Epoch 400: TotalLoss=0.083495 | Sup=0.083683 | Unsup=-0.187619 | TrainAcc=0.9615
Fold 1 Epoch 500: TotalLoss=0.043546 | Sup=0.043734 | Unsup=-0.188900 | TrainAcc=0.9872
Fold 1 Epoch 600: TotalLoss=0.080883 | Sup=0.081063 | Unsup=-0.180034 | TrainAcc=0.9615
Fold 1 Epoch 700: TotalLoss=0.028617 | Sup=0.028799 | Unsup=-0.182276 | TrainAcc=1.0000
Fold 1 Epoch 800: TotalLoss=0.018585 | Sup=0.018771 | Unsup=-0.186240 | TrainAcc=1.0000
Fold 1 Epoch 900: TotalLoss=0.036067 | Sup=0.036255 | Unsup=-0.188140 | TrainAcc=0.9744
Fold 1 Epoch 1000: TotalLoss=0.097045 | Sup=0.097235 | Unsup=-0.189781 | TrainAcc=0.9615
Fold 1 Epoch 1100